### Import of data

In [63]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

### Data we going to use in csv

In [64]:
features = ["hktv_ad_id", "order_value", "primary_category_name_en"]

### Import csv, from 2021 November to 2022 November, each months 1st day

In [65]:
csv_1 = pd.read_csv('csv/Public_Transaction_2021-11-01.csv', usecols=features)
csv_2 = pd.read_csv('csv/Public_Transaction_2021-12-01.csv', usecols=features)
csv_3 = pd.read_csv('csv/Public_Transaction_2022-01-01.csv', usecols=features)
csv_4 = pd.read_csv('csv/Public_Transaction_2022-02-01.csv', usecols=features)
csv_5 = pd.read_csv('csv/Public_Transaction_2022-03-01.csv', usecols=features)
csv_6 = pd.read_csv('csv/Public_Transaction_2022-04-01.csv', usecols=features)
csv_7 = pd.read_csv('csv/Public_Transaction_2022-05-01.csv', usecols=features)
csv_8 = pd.read_csv('csv/Public_Transaction_2022-06-01.csv', usecols=features)
csv_9 = pd.read_csv('csv/Public_Transaction_2022-07-01.csv', usecols=features)
csv_10 = pd.read_csv('csv/Public_Transaction_2022-08-01.csv', usecols=features)
csv_11 = pd.read_csv('csv/Public_Transaction_2022-09-01.csv', usecols=features)
csv_12 = pd.read_csv('csv/Public_Transaction_2022-10-01.csv', usecols=features)
csv_13 = pd.read_csv('csv/Public_Transaction_2022-11-01.csv', usecols=features)

### Joining all the csv into a large dataframe

In [66]:
df_1 = pd.concat([csv_1,csv_2,csv_3,csv_4,csv_5,csv_6,csv_7,csv_8,csv_9,csv_10,csv_11,csv_12,csv_13], ignore_index=True)

### Number of Data

In [67]:
df_1.shape[0]

3030730

### Read the user's data including gender and age range

In [68]:
f = open("sampleJSON.txt", encoding='utf-8').read()
feature = ["hktv_ad_id", "gender", "age_range"]
t = f.split("\n")
t.pop()
empty = []
for i in t:
    rows = json.loads(i)
    empty.append([rows["hktv_ad_id"], rows["gender"], rows["age_range"]])
df_2 = pd.DataFrame(empty, columns=feature)

### order value : From range (i.e. 5000-6000) to value (5000+6000 /2 = 5500)

In [69]:
#Make grouped data to numerical data

data = []
for i in range(len(df_1['order_value'])):
    num = df_1['order_value'][i].split(' - ')
    data.append((int(num[0])+int(num[1]))/2)

df_1["price"] = data

### same as order value, from age_range ot age

In [70]:
#Make grouped data to numerical data

data = []
for i in range(len(df_2['age_range'])):
    if df_2['age_range'][i] == None or df_2['age_range'][i] == "":
        data.append(np.nan)
        continue
    if df_2['age_range'][i].split('_')[1] == '55plus':
        data.append(60)
    elif df_2['age_range'][i].split('_')[1] == "12":
        data.append(10)
    else:
        num = df_2['age_range'][i].split('_')
        data.append((int(num[1])+int(num[2]))/2)

df_2['age'] = data

### Merging the transaction table and user data table, by the customer id (hktv_ad_id)

In [71]:
df = pd.merge(df_1, df_2, on="hktv_ad_id")
df.drop(labels=["order_value", "age_range"], axis=1, inplace=True)

In [72]:
#dealing with null and none data?

#price -> mean according to?
df['price'].fillna(value = df['price'].mean(), inplace = True)
#age -> mean according to?
df['age'].fillna(value = df['age'].mean(), inplace = True)
#gender = none
df.dropna(inplace=True)

In [73]:
#Check for the missing value again
df.isna().sum()

hktv_ad_id                  0
primary_category_name_en    0
price                       0
gender                      0
age                         0
dtype: int64

In [74]:
#Rename the 'primary_category_name_en' to 'category'
df.rename(columns={'primary_category_name_en':'category', 'hktv_ad_id':'user'}, inplace=True)
#Reorder to let the target value to be the last column
df = df.reindex(columns=["user", "gender", "age", "category", "price"])

In [75]:
from sklearn.preprocessing import LabelEncoder

In [76]:
l1 = LabelEncoder()
cat = ['category', 'gender']
for parameter in cat:
    l1.fit(df[parameter])
    df[parameter] = l1.transform(df[parameter])

In [77]:
# Number of data after merge and concating all csv
df.shape[0]

106446

In [78]:
df['age'] = df['age'].astype('int')
df['price'] = df['price'].astype('int')

In [79]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22,12,324
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,424
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,424
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,24
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,24


# Dimention Reduction JT <hr>

In [80]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22,12,324
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,424
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,424
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,24
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,24


In [81]:
from sklearn.preprocessing import StandardScaler

In [82]:
variables = ['category', 'gender', 'age']
x = df.loc[:, variables].values
y = df.loc[:,['price']].values
x = StandardScaler().fit_transform(x)
x = pd.DataFrame(x)

In [83]:
from sklearn.decomposition import PCA

In [84]:
pca = PCA()
x_pca = pca.fit_transform(x)
x_pca = pd.DataFrame(x_pca)
x_pca.head()

,0,1,2
0,-1.557824,-0.255189,-0.959979
1,-0.824967,-0.545002,-0.355248
2,-0.824967,-0.545002,-0.355248
3,-0.824967,-0.545002,-0.355248
4,-0.824967,-0.545002,-0.355248


### From Below, we can see that only two parameter cannot even capture 70% of the variance of the data, therefore we won't conduct any dimention reduction here.

In [85]:
explained_variance = pca.explained_variance_ratio_
explained_variance[:2].sum()

0.6859674795833697

In [86]:
x_pca['price']=y
x_pca.columns = ['category', 'gender', 'age', 'price']
x_pca.head()

,category,gender,age,price
0,-1.557824,-0.255189,-0.959979,324
1,-0.824967,-0.545002,-0.355248,424
2,-0.824967,-0.545002,-0.355248,424
3,-0.824967,-0.545002,-0.355248,24
4,-0.824967,-0.545002,-0.355248,24


# Regression Nathalie<hr>

## Thing's to do <br>
<h3><ol>
<li> train test split: Use sklearn library, 80% training 20% test</li>
<li> Model Training: use the best degree aka the best fit line</li>
<li> Performance Testing: Use R2 and RMS From HW 5</li>
</ol></h3>

In [87]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22,12,324
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,424
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,424
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,24
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30,12,24


In [88]:
print(len(df))

106446


For 3 predictors (), the equation includes all the possible combinations of different order polynomials. This is known as Multi-dimensional Polynomial Regression.

1. Import all libraries needed

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import math
import seaborn as sns

2. Split the dataframe into traing set and test set

In [90]:
X = df[['gender', 'age', 'category']]
Y = df['price']

df = df.sample(frac=1).reset_index(drop=True)# shuffle the data

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, train_size = .8)
# random_state is a pseudo-random number parameter that allows you to reproduce the same train test split each time you run the code. 

In [91]:
print(X)

        gender  age  category
8            0   22        12
17           0   30        12
18           0   30        12
19           0   30        12
20           0   30        12
...        ...  ...       ...
175569       1   30        13
175570       1   30        13
175571       0   30        13
175572       0   30        13
175573       0   30        13

[106446 rows x 3 columns]


3. Build the Regression Model
   
    3.1. Multiple linear Regression

In [92]:
mlr = LinearRegression()
mlr.fit(x_train, y_train)

LinearRegression()

In [93]:
# model equation of multiple linear regression
print("Intercept: " ,mlr.intercept_)
print("Coefficients:")
list(zip(x, mlr.coef_))

Intercept:  998.9020068373156
Coefficients:


[(0, -20.43775600018533), (1, -2.6556650572299763), (2, -12.95881807890112)]

In [94]:
# prediction of test set
y_pred_mlr= mlr.predict(x_test)
print("Prediction for test set: {}".format(y_pred_mlr))

Prediction for test set: [739.82525266 739.82525266 854.11845752 ... 710.61293703 793.84572566
 730.01015689]


In [95]:
# Since it is difficult to plot out a graph with 4 parameter, so I will only plot out the graph comparsion between the actual value and predicted value
# Actual values and the predicted values
mlr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_pred_mlr})
mlr_diff.head()

,Actual value,Predicted value
130357,574,739.825253
75652,1149,739.825253
80962,374,854.118458
159309,424,739.825253
172962,374,802.602692


 Evaluation of regression model

In [96]:
score_mlr = mlr.score(x_test, y_test)
print(score_mlr)

0.0013531438081397784


 3. Build the Regression Model
 
    3.2 LASSO Regression Model

    LASSO Regression comes in handy when there is too many features, some of them are can be completely eliminated by setting the coefficients to zero, known as "L1-norm 
    Regularization".

In [97]:
from sklearn.linear_model import Lasso

In [98]:
# define model
# create an instance of Lasso Regression implementation
model = Lasso(alpha=1.0)
# fit the Lasso model
model.fit(x_train, y_train)
# make prediction
y_pred_lasso = model.predict(x_test)

 Evaluation of regression model

In [99]:
model.score(x_test,y_test)

0.0013730569593132236

3. Build the Regression Model

    3.3 Random Forest

# KNN Tree

In [100]:
from sklearn.neighbors import KNeighborsRegressor

In [101]:
knn_model = KNeighborsRegressor(n_neighbors=3)

In [102]:
knn_model.fit(x_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [103]:
from sklearn.metrics import mean_squared_error
from math import sqrt
train_preds = knn_model.predict(x_train)
mse = mean_squared_error(y_train, train_preds)
print(mse)

751922.0174672888


# Support Vector Machine

In [104]:
x_train

,gender,age,category
130037,0,50,12
108356,1,30,12
13771,0,30,12
84125,0,35,12
84894,1,60,8
...,...,...,...
33357,1,50,12
74718,0,50,12
69808,0,35,12
71366,1,35,12


In [105]:
from sklearn.svm import SVC

In [106]:
clf = SVC(kernel = "linear")
clf.fit(x_train[:1000], y_train[:1000])
y_pred_svm = clf.predict(x_test)

In [107]:
mse1 = mean_squared_error(y_test, y_pred_svm)
print(mse1)

799872.1524189761


In [108]:
clf = SVC(kernel = "poly")
clf.fit(x_train[:1000], y_train[:1000])
y_pred_svm = clf.predict(x_test)

In [109]:
mse2 = mean_squared_error(y_test, y_pred_svm)
print(mse2)

798504.02184124


In [110]:
clf = SVC(kernel = "rbf")
clf.fit(x_train[:1000], y_train[:1000])
y_pred_svm = clf.predict(x_test)

In [111]:
mse3 = mean_squared_error(y_test, y_pred_svm)
print(mse3)

798504.02184124


In [112]:
clf = SVC(kernel = "sigmoid")
clf.fit(x_train[:1000], y_train[:1000])
y_pred_svm = clf.predict(x_test)

In [113]:
mse4 = mean_squared_error(y_test, y_pred_svm)
print(mse4)

812961.8658994833


In [114]:
print(f"linear:{mse1}, polynomial:{mse2}, rbf:{mse3}, signmoid:{mse4}")

linear:799872.1524189761, polynomial:798504.02184124, rbf:798504.02184124, signmoid:812961.8658994833


# Decision Tree   Joseph<hr>

In [115]:
df.head()

,user,gender,age,category,price
0,nBJBerFiYaJ1sBfKzkUBOjuW6X4aOqwEWQ3R4Daoj1M=,0,35,12,1149
1,rAT3MP0u2FOHvh5QN5eQEGO7TWSJXs0F3LCJzPeez4A=,1,22,12,24
2,kMpDqntoAC2PPrexSfehgZrLejaadQG24yrDQ/dI1/0=,0,35,12,874
3,LLcIqOYmxfG3KhPyOACoc5LVB1yLDftUn8A6BbL2bgU=,0,30,4,224
4,bPhBGfv4sLFBFwjuG5XoqQjpQnOaWhAHqexT87UkDMg=,0,39,12,874


## Thing's to do <br>
<h3><ol>
<li> train test split: Use sklearn library, 80% training 20% test</li>
<li> Model Training</li>
<li> Performance Testing: Use R2 and RMS From HW 5</li>
</ol></h3>

# Visuallization (HW4) <hr>

## Thing's to do <br>
<h3><ol>
<li>Visualize the dimention reduction data</li>
<li>Visualize the regression line with data point</li>
</ol></h3>